In [1]:
###USING 3 YEARS OF ASEC MARCH DATA
###ENSURE TO GROUP BY SPM UNIT AND YEAR

import numpy as np
import pandas as pd
import microdf as mdf

In [2]:
raw = pd.read_csv("cps.csv.gz")

In [3]:
#VAR LIST: MONTH	CPSID	ASECFLAG	ASECWTH	STATEFIP	COUNTY	PERNUM	CPSIDP	ASECWT	AGE	SEX	RACE	
#MARST	VETSTAT	HISPAN	INCTOT	SPMTOTRES	SPMTHRESH	SPMFAMUNIT	MIGSTA1

person = raw.copy(deep=True)

#Calculate Weight
person['w'] = person.ASECWT/100

#Calculate People in Poverty based on SPM Threshold
person['Poor'] = person.SPMTOTRES < person.SPMTHRESH

person.INCTOT.replace({999999999 : 0 }, inplace = True)

#Ignore non-CA residents
person.drop(person[person['STATEFIP'] != 6].index, inplace = True) 




#Gen variable with county name

person['COUNTY_NAME'] = person['COUNTY'].map({ '001': 'Alameda', '007': 'Butte', '013': 'Contra Costa', '017': 'El Dorado', '019': 'Fresno', '023': 'Humboldt', '025': 'Imperial', '029': 'Kern', '031': 'Kings',
                                              '037': 'Los Angeles', '039': 'Madera', '041': 'Marin', '047': 'Merced', '053': 'Monterey', '055': 'Napa', '059': 'Orange', '061': 'Placer', '065': 'Riverside',
                                              '067': 'Sacramento', '071': 'San Bernardino', '073': 'San Diego', '075': 'San Francisco', '077': 'San Joaquin', '079': 'San Luis Obispo', '081': 'San Mateo', '083':
                                              'Santa Barbara', '087': 'Santa Cruz', '089': 'Shasta', '095': 'Solano', '097': 'Sonoma', '099': 'Stanislaus', '107': 'Tulare', '111': 'Ventura', '113': 'Yolo'})



#Calculating median income by county

person = mdf.MicroDataFrame(person, weights = "ASECWT")

county = person[person.INCTOT > 0].groupby('COUNTY')[['INCTOT']].median().reset_index()

county.rename(columns = {'INCTOT' : 'MED_COUNTY_INC'}, inplace = True)

#county

In [4]:
person2 = mdf.MicroDataFrame(person.merge(county, on = 'COUNTY'), weights = "ASECWT")


In [5]:
#Define the eligible as being 18 or older and making less than two times their county's median income

person2['ELIGIBLE'] = np.where((person2['AGE'] >= 18) & (person2['INCTOT'] < (2 * person2['MED_COUNTY_INC'])) & 
                                                         (person2['MIGSTA1'] != 6),1,0)

In [6]:
spmu =  person2.groupby(['YEAR', 'SPMFAMUNIT'])[['ELIGIBLE']].sum().reset_index()
spmu.rename(columns = {'ELIGIBLE' : 'SPMU_ELIGIBLE'}, inplace = True)

In [7]:
person3 = mdf.MicroDataFrame(person2.merge(spmu, on = ['YEAR', 'SPMFAMUNIT']), weights = 'ASECWT')

In [8]:
person3['SPMTOTRESNEW'] = person3['SPMTOTRES'] + 12000 * person3.SPMU_ELIGIBLE

In [9]:
person3.poverty_rate('SPMTOTRESNEW', 'SPMTHRESH')

0.036636156861466

In [10]:
person3.poverty_rate('SPMTOTRES', 'SPMTHRESH')

0.17417376602940388

In [11]:
#Total Cost
Eligible_Residents = person2['ELIGIBLE'].sum()

Cost_Per_Resident = 12000

Total = ((Eligible_Residents * Cost_Per_Resident)/1e9)/3

Total


0.11492799999999999

In [12]:
(person3.INCTOT == 2055999).sum()

3258.68